<a href="https://colab.research.google.com/github/victorchall/EveryDream/blob/main/EveryDream_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Please read the documentation here before you start.

I suggest reading this doc before you connect to your runtime to avoid using credits or being charged while you figure it out.

[Auto Captioning Readme](doc/AUTO_CAPTION.md)

This notebook requires an Nvidia GPU instance. Any will do, you don't need anything power.  As low as 4GB should be fine.

Only colab has automatic file transfers at this time.  If you are using another platform, you will need to manually download your output files.

In [ ]:
#download repo
!git clone https://github.com/victorchall/EveryDream.git
# Set working directory
%cd EveryDream

In [ ]:
# install requirements
!pip install torch=='1.12.1+cu113' 'torchvision==0.13.1+cu113' --extra-index-url https://download.pytorch.org/whl/cu113
!pip install pandas>='1.3.5'
!git clone https://github.com/salesforce/BLIP scripts/BLIP
!pip install timm
!pip install fairscale=='0.4.4'
!pip install transformers=='4.19.2'
!pip install timm
!pip install aiofiles

#Extract Frames from video

Here we will use the folder input_vid and upload in the same way we did our images

In [ ]:
!python /scripts/extract_video_frames.py \
--vid_dir input_vid \
--out_dir output/vid \
--format png \
--interval 10 

Move the extracted frames to the input directory for captions

In [ ]:
!cp -r output/vid input

# Upload your input images into the EveryDream/input folder

![upload to input](https://github.com/victorchall/EveryDream/blob/main/demo/upload_images_caption.png?raw=1)

## Please read the documentation for information on the parameters

[Auto Captioning](doc/AUTO_CAPTION.md)

*You cannot have commented lines between uncommented lines.  If you uncomment a line below, move it above any other commented lines.*

*!python must remain the first line.*

Default params should work fairly well.

In [ ]:
!python scripts/auto_caption.py \
--img_dir input \
--out_dir output \
#--format mrwho \
#--min_length 34 \
#--q_factor 1.3 \
#--nucleus \

## mutiple files can be targeted in succession

#!python scripts/auto_caption.py \
#--img_dir input/subfolder \
#--out_dir output/subfolder \
#--format mrwho \
#--min_length 34 \
#--q_factor 1.3 \
#--nucleus \

# Laion Downloader

* --laion_dir: directory with laion parquet files, default is ./laion

* --search_text: csv of words with AND logic, ex \"photo,man,dog\"

* --out_dir: directory to download files to, ive defaulted this to inputs so they can be captioned 

* --log_dir: directory for logs, if ommitted will not log, logs may be large!

* --column:column to search for matches, defaults is 'TEXT', but you could use 'URL' if you wanted",

* --limit: max number of matching images to download, warning: may be slightly imprecise due to concurrency and http errors, defaults is 100

* --min_hw: min height AND width of image to download, default is 512
  
* --force: forces a full download of all images, even if no search is provided, USE CAUTION!

* --parquet_skip: skips the first n parquet files on disk, useful to resume
        
* --verbose: additional logging of URL and TEXT 
        
* --test: skips downloading, for checking filters, use with "--verbose"


In [ ]:
!python scripts/download_laion.py \
--laion_dir ./laion \
--search_text "photo,man,dog" \
#--out_dir input \
#--log_dir logs \
#--column TEXT \
#--limit 100 \
#--min_hw 512 \
#--force False \
#--parquet_skip 0 \
#--Verbose False \
#--test not \


Here we can take our now captioned images and replace generic terms with our subjects

* --find: will search for a word in this case man

* --replace: will replace our found word with in this case bob smith

* --append_only: this will allow us to add a tag at he end 

In [ ]:
!python scripts/filename_replace.py \
--img_dir output \
--find "man" \
--replace "bob smith"

Now we can chose to create text files based on our file names, this is usefull for images with very long discriptions or tag list, windows has a limit of 256 characters, and files will not transfer correctly to a windows program if they are longer, moving these files in a zip is fine however and causes no issues


In [ ]:
!python scripts/createtxtfromfilename.py

Compress our images 

In [ ]:
/content/EveryDream/scripts/compress_img.py \
--img_dir output \
--out_dir output/compressed_images \
--max_mp 1.5 
#--overwrite False \
#--Quality 95  \
#--noresize False \
#--delete \

## Download your DataSet from EveryDream/output

If you're on a colab you can use the cell below to push your output to your Gdrive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir /content/drive/MyDrive/Auto_Data_sets
!cp -r output/ /content/drive/MyDrive/Auto_Data_sets

## If not on colab/gdrive, the following will zip up your files for extraction

You'll still need to use your runtime's own download feature to download the zip.

![output zip](https://github.com/victorchall/EveryDream/blob/main/demo/output_zip.png?raw=1)

In [ ]:
!pip install patool

import patoolib

!mkdir output/zip

!zip -r output/zip/output.zip output